In [1]:
import pandas as pd 
import numpy as np  
import re
import random
import json 

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time

path = 'C:/Users/GMEU0170/OneDrive - Hyundai Motor Europe/GME_CD/Documents/2.__Code/_.py/web_scraping/selenium/'

In [2]:
# Set up Chrome options (headless mode is optional)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [3]:
# Use webdriver-manager to install and manage ChromeDriver
service = Service(ChromeDriverManager().install())

ConnectionError: Could not reach host. Are you offline?

### Scraping data

In [4]:
darmas_chrome = pd.read_csv('output/chrome_scraping/darmas_chrome_scrape.csv')

darmas_crawled = pd.read_csv('output/darmas/final_df_transformed.csv')

In [5]:
to_crawl = ['https://darmas.de/fahrzeugmarkt/details/67375b9ce9b9ca112257b30c',
        'https://darmas.de/fahrzeugmarkt/details/67375b17e9b9ca1122567752',
        "https://darmas.de/fahrzeugmarkt/details/67efd37e8aafb529779c9604"]

to_crawl = [i for i in darmas_chrome[darmas_chrome.columns[0]] if i not in list(darmas_crawled['url'])]

len(to_crawl)

28

In [6]:
# Standardized headings for classification (used in the processing phase)
headings = [
    "Innenausstattung", "Lenkrad", "Räder & Fahrwerk",
    "Multimedia", "Assistenzsysteme", "Einparkhilfe",
    "Sitze", "Extras", "Sicherheit & Umwelt",
    "Licht & Scheinwerfer", "Airbags", "Emissionen", "Verbrauch & Umwelt",
    "Eigenschaften", "Kosten Modell"
]

# User-Agent list for rotation
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.1 Safari/605.1.15",
    # Add more user agents as needed
]

# Files for raw and processed data
raw_output_file = "output/darmas/raw_data.jsonl"
processed_output_file = "output/darmas/processed_data.jsonl"

# Batch size (create a new driver every <batch_size> URLs)
# batch_size = 20

# List of URLs to scrape (this should be defined)
# urls = [i for i in darmas[darmas.columns[0]][:10]]


In [7]:
def get_driver(service):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--blink-settings=imagesEnabled=false")
    ua = random.choice(user_agents)
    chrome_options.add_argument(f"user-agent={ua}")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    # Explicitly set the page load strategy to "normal"
    chrome_options.page_load_strategy = "normal"
    
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    driver.implicitly_wait(10)
    return driver

In [ ]:
driver = get_driver(service)

In [ ]:
import json

output_file = "output/darmas/optimized_records_test2204.jsonl"
AUSSTATTUNG_TAB_XPATH = "//ul[@class='nav nav-tabs']//a[normalize-space(text())='Ausstattung']"
AUSSTATTUNG_CSS = ".col.text-truncate"

with open(output_file, "w", encoding="utf-8") as fout:  # Open once.
    for i, url in enumerate(bugs):
        try:
            driver.get(url)
            ausstattung_tab = driver.find_element(By.XPATH, AUSSTATTUNG_TAB_XPATH)
            driver.execute_script("arguments[0].click();", ausstattung_tab)
            ausstattungs_elements = driver.find_elements(By.CSS_SELECTOR, AUSSTATTUNG_CSS)
            ausstattung_raw = [el.text.strip() for el in ausstattungs_elements if el.text.strip()]
    
            # Build record.
            row = {"url": url, "ausstattung": ausstattung_raw}
    
            # Write the record as a JSON line.
            fout.write(json.dumps(row, ensure_ascii=False) + "\n")
            fout.flush()
    
            print(f"Finished link {i}")
    
        except Exception as e:
            print(f"Error scraping URL {url}: {e}")
            continue


In [ ]:
# df = pd.DataFrame(records,columns=['url','ausstellung'])
df = pd.read_json('output/darmas/optimized_records_test2204.jsonl', lines=True)

df

In [ ]:
df.to_csv('output/darmas/optimized_records_test2204.csv',index=False)

In [ ]:
js = pd.read_csv("output/darmas/optimized_records.csv")

df = js.copy()

columns_to_expand = ['ausstattung']

dummy_dfs = []

for col in columns_to_expand:

    df[col] = df[col].apply(lambda x: ','.join([item.strip() for item in str(x).split(',') if item.strip()]))
    dummy = df[col].str.get_dummies(sep=',')
    
    dummy = dummy.astype(bool)
    
    dummy_dfs.append(dummy)

# Merge all dummy dataframes horizontally
dummy_df = pd.concat(dummy_dfs, axis=1)

# Optionally, you can join these dummy columns back with your original DataFrame.
final_df = pd.concat([df, dummy_df], axis=1)
final_df.columns = final_df.columns.str.replace("'", '', regex=False)

final_df.to_csv("output/darmas/optimized_records.csv",index=False)